# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=5


# Trade profile

Choose country and years of interest

In [4]:
country_of_interest = comtrade.m49_stome_principe
years_of_interest = comtrade.year_range(2021, 2022)   
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Sao Tome and Principe 2021


### Global exports and imports

In [5]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


reporterDesc partnerDesc flowDesc  \
refYear flowCode                                               
2021    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   

                 primaryValueFormated  
refYear flowCode                       
2021    M              166,275,006.87  
        X               19,174,845.70

## Principais parceiros comerciais
### Principais destinos das exportações

In [6]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    rank_filter=show_ony_top)

In [7]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated']
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner'])

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated
75,Sao Tome and Principe,2021,Export,1,Netherlands,"8,641,395.94"
92,Sao Tome and Principe,2021,Export,2,Belgium,"4,077,334.46"
44,Sao Tome and Principe,2021,Export,3,Portugal,"2,511,956.44"
105,Sao Tome and Principe,2021,Export,4,Germany,"1,735,756.74"
115,Sao Tome and Principe,2021,Export,5,Cameroon,"812,274.41"
97,Sao Tome and Principe,2021,Export,6,Senegal,"471,298.30"
71,Sao Tome and Principe,2021,Export,7,France and Monaco,"239,179.43"
29,Sao Tome and Principe,2021,Export,8,Gabon,"142,582.50"
9,Sao Tome and Principe,2021,Export,9,Spain,"120,628.77"
25,Sao Tome and Principe,2021,Export,10,Angola,"91,310.96"


# Exportações

## Principais produtos exportados

In [8]:
show_only_top=20
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='X',
                                           motCode=0,
                                           rank_filter=show_only_top)


In [9]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        comtrade.PERC_CMD_IN_PARTNER]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,perc_cmd_for_partner
210,Sao Tome and Principe,2021,Export,1,18,Cocoa and cocoa preparations,"11,598,030.93",60.49%
24,Sao Tome and Principe,2021,Export,2,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","6,312,475.49",32.92%
44,Sao Tome and Principe,2021,Export,3,88,"Aircraft, spacecraft, and parts thereof","457,474.43",2.39%
149,Sao Tome and Principe,2021,Export,4,09,"Coffee, tea, mate and spices","185,471.29",0.97%
57,Sao Tome and Principe,2021,Export,5,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","150,143.90",0.78%
95,Sao Tome and Principe,2021,Export,6,73,Iron or steel articles,"77,291.62",0.40%
172,Sao Tome and Principe,2021,Export,7,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","55,016.28",0.29%
115,Sao Tome and Principe,2021,Export,8,08,"Fruit and nuts, edible; peel of citrus fruit or melons","51,911.38",0.27%
41,Sao Tome and Principe,2021,Export,9,30,Pharmaceutical products,"44,254.84",0.23%
146,Sao Tome and Principe,2021,Export,10,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","42,244.13",0.22%


## Principais produtos exportados e destinos

In [10]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0, 
                                           extra_cols=[],
                                           rank_filter=show_only_top)

In [11]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
896,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,1,Netherlands,"5,821,899.06",67.37%,50.20%
213,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,2,Belgium,"3,498,677.74",85.81%,30.17%
557,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,4,Germany,"1,735,464.83",99.98%,14.96%
133,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,3,Portugal,"127,662.98",5.08%,1.10%
117,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,9,Spain,"120,628.77",100.00%,1.04%
924,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,11,Switzerland,"84,440.07",100.00%,0.73%
516,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,8,Gabon,"64,850.93",45.48%,0.56%
519,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,13,Italy,"49,327.94",100.00%,0.43%
515,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,7,France and Monaco,"48,875.15",20.43%,0.42%
798,Sao Tome and Principe,2021,Export,1,Cocoa and cocoa preparations,14,United States of America,"36,871.39",81.88%,0.32%


## Principais **parceiros** das exportações e produtos exportados

In [12]:
show_only_top = 5
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [14]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner',	'partnerDesc','rank_cmd', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]
export_partners[cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, 
                                                                                       comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
896,Sao Tome and Principe,2021,Export,1,Netherlands,1,Cocoa and cocoa preparations,"5,821,899.06",67.37%,50.20%
642,Sao Tome and Principe,2021,Export,1,Netherlands,2,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","2,819,496.88",32.63%,44.67%
213,Sao Tome and Principe,2021,Export,2,Belgium,1,Cocoa and cocoa preparations,"3,498,677.74",85.81%,30.17%
326,Sao Tome and Principe,2021,Export,2,Belgium,2,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","526,142.51",12.90%,8.33%
877,Sao Tome and Principe,2021,Export,2,Belgium,8,"Fruit and nuts, edible; peel of citrus fruit or melons","21,390.77",0.52%,41.21%
214,Sao Tome and Principe,2021,Export,2,Belgium,17,"Food industries, residues and wastes thereof; prepared animal fodder","12,231.28",0.30%,100.00%
274,Sao Tome and Principe,2021,Export,2,Belgium,10,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","11,915.54",0.29%,28.21%
410,Sao Tome and Principe,2021,Export,3,Portugal,2,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","2,197,706.64",87.49%,34.82%
133,Sao Tome and Principe,2021,Export,3,Portugal,1,Cocoa and cocoa preparations,"127,662.98",5.08%,1.10%
347,Sao Tome and Principe,2021,Export,3,Portugal,5,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","62,457.94",2.49%,41.60%


# Importações

## Principais produtos importados

In [17]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [18]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
2459,Sao Tome and Principe,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","32,665,021.66",World,19.65%
1621,Sao Tome and Principe,2021,Import,2,22,"Beverages, spirits and vinegar","13,283,400.99",World,7.99%
2120,Sao Tome and Principe,2021,Import,3,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","11,123,581.21",World,6.69%
371,Sao Tome and Principe,2021,Import,4,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","10,211,442.57",World,6.14%
1610,Sao Tome and Principe,2021,Import,5,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","9,301,857.40",World,5.59%
170,Sao Tome and Principe,2021,Import,6,10,Cereals,"7,986,390.46",World,4.80%
1439,Sao Tome and Principe,2021,Import,7,11,"Products of the milling industry; malt, starches, inulin, wheat gluten","5,889,068.87",World,3.54%
174,Sao Tome and Principe,2021,Import,8,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement","5,449,316.95",World,3.28%
820,Sao Tome and Principe,2021,Import,9,02,Meat and edible meat offal,"4,140,041.32",World,2.49%
1445,Sao Tome and Principe,2021,Import,10,39,Plastics and articles thereof,"4,008,572.38",World,2.41%


## Principais produtos importados e origens

In [21]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [25]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
5448,Sao Tome and Principe,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,Angola,"15,251,564.08",76.63%,46.69%
1519,Sao Tome and Principe,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,Togo,"14,673,794.22",94.18%,44.92%
5531,Sao Tome and Principe,2021,Import,2,"Beverages, spirits and vinegar",1,Portugal,"11,780,920.34",14.39%,88.69%
1387,Sao Tome and Principe,2021,Import,3,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",1,Portugal,"6,367,300.70",7.78%,57.24%
3416,Sao Tome and Principe,2021,Import,3,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",4,China,"1,649,026.75",13.82%,14.82%
7535,Sao Tome and Principe,2021,Import,3,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",7,Belgium,"1,620,198.78",50.05%,14.57%
5018,Sao Tome and Principe,2021,Import,4,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",1,Portugal,"3,660,341.65",4.47%,35.85%
6313,Sao Tome and Principe,2021,Import,4,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",13,Gabon,"1,880,921.65",97.50%,18.42%
6079,Sao Tome and Principe,2021,Import,4,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",4,China,"1,744,249.70",14.62%,17.08%
1961,Sao Tome and Principe,2021,Import,4,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",5,Japan,"662,830.27",13.01%,6.49%


### Principal origem das importações

In [26]:
show_ony_top=10
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [27]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated']
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner'])

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated
425,Sao Tome and Principe,2021,Import,1,Portugal,"81,894,455.08"
599,Sao Tome and Principe,2021,Import,2,Angola,"19,902,136.65"
219,Sao Tome and Principe,2021,Import,3,Togo,"15,580,979.59"
29,Sao Tome and Principe,2021,Import,4,China,"11,933,665.74"
643,Sao Tome and Principe,2021,Import,5,Japan,"5,093,755.45"
407,Sao Tome and Principe,2021,Import,6,Turkey,"3,475,302.33"
345,Sao Tome and Principe,2021,Import,7,Belgium,"3,237,077.40"
178,Sao Tome and Principe,2021,Import,8,United States of America,"2,954,367.93"
18,Sao Tome and Principe,2021,Import,9,Brazil,"2,821,190.34"
181,Sao Tome and Principe,2021,Import,10,Spain,"2,380,679.09"


## Principais **parceiros** das importações e produtos importados

In [41]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [42]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
5531,Sao Tome and Principe,2021,Import,1,Portugal,"Beverages, spirits and vinegar","11,780,920.34",14.39%,88.69%
1387,Sao Tome and Principe,2021,Import,1,Portugal,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","6,367,300.70",7.78%,57.24%
8214,Sao Tome and Principe,2021,Import,1,Portugal,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","5,583,256.51",6.82%,60.02%
3624,Sao Tome and Principe,2021,Import,1,Portugal,"Products of the milling industry; malt, starches, inulin, wheat gluten","5,205,456.25",6.36%,88.39%
5018,Sao Tome and Principe,2021,Import,1,Portugal,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","3,660,341.65",4.47%,35.85%
7727,Sao Tome and Principe,2021,Import,1,Portugal,"Food industries, residues and wastes thereof; prepared animal fodder","3,093,648.06",3.78%,99.66%
2344,Sao Tome and Principe,2021,Import,1,Portugal,"Dairy produce; birds' eggs; natural honey; edible products of animal origin, not elsewhere specified or included","3,070,574.65",3.75%,82.09%
4958,Sao Tome and Principe,2021,Import,1,Portugal,Plastics and articles thereof,"3,042,181.18",3.71%,75.89%
2934,Sao Tome and Principe,2021,Import,1,Portugal,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","2,895,712.72",3.54%,79.26%
1314,Sao Tome and Principe,2021,Import,1,Portugal,"Preparations of vegetables, fruit, nuts or other parts of plants","2,420,894.08",2.96%,94.30%


## Análise de simetrias nos dados

In [43]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [46]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(20).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
75,Sao Tome and Principe,Netherlands,World,2021,Export,"8,641,395.94",8641395.939000,45.07%,All modes of transport,True,All customs procedure codes
92,Sao Tome and Principe,Belgium,World,2021,Export,"4,077,334.46",4077334.457000,21.26%,All modes of transport,True,All customs procedure codes
44,Sao Tome and Principe,Portugal,World,2021,Export,"2,511,956.44",2511956.444000,13.10%,All modes of transport,True,All customs procedure codes
105,Sao Tome and Principe,Germany,World,2021,Export,"1,735,756.74",1735756.738000,9.05%,All modes of transport,True,All customs procedure codes
115,Sao Tome and Principe,Cameroon,World,2021,Export,"812,274.41",812274.412000,4.24%,All modes of transport,True,All customs procedure codes
97,Sao Tome and Principe,Senegal,World,2021,Export,"471,298.30",471298.300000,2.46%,All modes of transport,True,All customs procedure codes
71,Sao Tome and Principe,France and Monaco,World,2021,Export,"239,179.43",239179.432000,1.25%,All modes of transport,True,All customs procedure codes
29,Sao Tome and Principe,Gabon,World,2021,Export,"142,582.50",142582.498000,0.74%,All modes of transport,True,All customs procedure codes
9,Sao Tome and Principe,Spain,World,2021,Export,"120,628.77",120628.775000,0.63%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [55]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=None,
                     motCode = 0,
                     customsCode=None,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(150).style.format({'perc': '{:,.2%}'.format}) 

/Users/jrc/develop/cipf-comtrade/comtrade.py:377: UserWarning: Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details
  warnings.warn("Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details")


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
1,Angola,Sao Tome and Principe,World,2021,Import,"142,609.80",142609.803000,0.30%,All modes of transport,True,All customs procedure codes
6,Armenia,Sao Tome and Principe,World,2021,Import,"3,375.51",3375.510000,0.01%,All modes of transport,True,All customs procedure codes
47,Armenia,Sao Tome and Principe,Sao Tome and Principe,2021,Import,"3,375.51",3375.510000,0.01%,All modes of transport,True,All customs procedure codes
193,"Asia, not elsewhere specified",Sao Tome and Principe,World,2021,Import,"10,777.97",10777.968000,0.02%,All modes of transport,True,All customs procedure codes
3,Australia,Sao Tome and Principe,World,2021,Import,"62,114.11",62114.110000,0.13%,All modes of transport,True,All customs procedure codes
4,Austria,Sao Tome and Principe,World,2021,Import,"10,002.43",10002.435000,0.02%,All modes of transport,True,All customs procedure codes
5,Bahrain,Sao Tome and Principe,World,2021,Import,"47,519.73",47519.734000,0.10%,All modes of transport,True,All customs procedure codes
7,Belgium,Sao Tome and Principe,World,2021,Import,"352,689.26",352689.257000,0.74%,All modes of transport,True,All customs procedure codes
8,Bolivia,Sao Tome and Principe,World,2021,Import,"2,029.17",2029.170000,0.00%,All modes of transport,True,All customs procedure codes
10,Brazil,Sao Tome and Principe,World,2021,Import,"28,786.00",28786.000000,0.06%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [104]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['Portugal']
flowCode = 'M'
cmd_detail = 'AG4'
show_only_top = 10

cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [105]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [106]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    cols = cols + ['perc']
    print(f"Importações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} por {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print(f"Não há importações de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
    result = ''

result

Importações de produtos de Portugal por Sao Tome and Principe reportados por Sao Tome and Principe


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc,perc
322,Sao Tome and Principe,Portugal,World,2020,Import,"Wine of fresh grapes, including fortified wines; grape must other than that of heading no. 2009","4,302,047.44",4302047.444000,5.93%,All modes of transport,True,All customs procedure codes,5.93%
162,Sao Tome and Principe,Portugal,World,2020,Import,Wheat or meslin flour,"3,357,366.76",3357366.763000,4.63%,All modes of transport,True,All customs procedure codes,4.63%
218,Sao Tome and Principe,Portugal,World,2020,Import,"Soya-bean oil and its fractions; whether or not refined, but not chemically modified","2,602,891.20",2602891.201000,3.59%,All modes of transport,True,All customs procedure codes,3.59%
1650,Sao Tome and Principe,Portugal,World,2020,Import,Electric generating sets and rotary converters,"2,593,166.72",2593166.720000,3.58%,All modes of transport,True,All customs procedure codes,3.58%
318,Sao Tome and Principe,Portugal,World,2020,Import,"Waters, including mineral and aerated waters, containing added sugar or sweetening matter, flavoured; other non-alcoholic beverages, not including fruit, nut or vegetable juices of heading no. 2009","2,382,298.18",2382298.181000,3.29%,All modes of transport,True,All customs procedure codes,3.29%
341,Sao Tome and Principe,Portugal,World,2020,Import,Preparations of a kind used in animal feeding,"2,240,970.03",2240970.031000,3.09%,All modes of transport,True,All customs procedure codes,3.09%
158,Sao Tome and Principe,Portugal,World,2020,Import,Rice,"1,886,718.17",1886718.165000,2.60%,All modes of transport,True,All customs procedure codes,2.60%
1693,Sao Tome and Principe,Portugal,World,2020,Import,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other apparatus for the transmission or reception of voice, images or other data (including wired/wireless networks), excluding items of 8443, 8525, 8527, or 8528","1,524,411.34",1524411.339000,2.10%,All modes of transport,True,All customs procedure codes,2.10%
369,Sao Tome and Principe,Portugal,World,2020,Import,"Portland cement, aluminous cement (ciment fondu), slag cement, supersulphate cement and similar hydraulic cements, whether or not coloured or in the form of clinkers","1,415,506.17",1415506.170000,1.95%,All modes of transport,True,All customs procedure codes,1.95%
269,Sao Tome and Principe,Portugal,World,2020,Import,"Pasta; whether or not cooked or stuffed with meat or other substance, or otherwise prepared, egg spaghetti, macaroni, noodles, lasagne, gnocchi, ravioli, cannelloni; couscous, whether or not prepared","1,303,813.78",1303813.777000,1.80%,All modes of transport,True,All customs procedure codes,1.80%


In [108]:
# invertemos

print(years_of_interest, partnerA, partnerB)
print(f"Exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

if flowCode == 'M':
    flowCodeInv = 'X'
else:
    flowCodeInv = 'M'

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    cols = cols + ['perc']
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print(f"Não há exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")
    result = ''
result

2021 678 620
Exportações de produtos de Portugal para Sao Tome and Principe reportados por Portugal


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc,perc,perc,perc
333,Portugal,Sao Tome and Principe,World,2021,Export,"Wine of fresh grapes, including fortified wines; grape must other than that of heading no. 2009","4,466,237.94",4466237.938000,6.66%,All modes of transport,True,All customs procedure codes,6.66%,6.66%,6.66%
157,Portugal,Sao Tome and Principe,World,2021,Export,Wheat or meslin flour,"2,976,638.02",2976638.020000,4.44%,All modes of transport,True,All customs procedure codes,4.44%,4.44%,4.44%
326,Portugal,Sao Tome and Principe,World,2021,Export,"Waters, including mineral and aerated waters, containing added sugar or sweetening matter, flavoured; other non-alcoholic beverages, not including fruit, nut or vegetable juices of heading no. 2009","2,554,882.29",2554882.288000,3.81%,All modes of transport,True,All customs procedure codes,3.81%,3.81%,3.81%
353,Portugal,Sao Tome and Principe,World,2021,Export,Preparations of a kind used in animal feeding,"2,215,309.27",2215309.266000,3.30%,All modes of transport,True,All customs procedure codes,3.30%,3.30%,3.30%
1548,Portugal,Sao Tome and Principe,World,2021,Export,"Automatic data processing machines and units thereof, magnetic or optical readers, machines for transcribing data onto data media in coded form and machines for processing such data, not elsewhere specified or included","2,013,626.02",2013626.022000,3.00%,All modes of transport,True,All customs procedure codes,3.00%,3.00%,3.00%
209,Portugal,Sao Tome and Principe,World,2021,Export,"Soya-bean oil and its fractions; whether or not refined, but not chemically modified","1,733,417.10",1733417.102000,2.58%,All modes of transport,True,All customs procedure codes,2.58%,2.58%,2.58%
330,Portugal,Sao Tome and Principe,World,2021,Export,Beer made from malt,"1,479,860.05",1479860.051000,2.21%,All modes of transport,True,All customs procedure codes,2.21%,2.21%,2.21%
91,Portugal,Sao Tome and Principe,World,2021,Export,"Vegetables, leguminous; shelled, whether or not skinned or split, dried","1,405,591.06",1405591.056000,2.10%,All modes of transport,True,All customs procedure codes,2.10%,2.10%,2.10%
626,Portugal,Sao Tome and Principe,World,2021,Export,"Reagents; diagnostic or laboratory reagents on a backing, prepared diagnostic or laboratory reagents whether or not on a backing, whether or not put up in the form of kits, other than those of heading 30.06; certified reference materials","1,219,019.69",1219019.686000,1.82%,All modes of transport,True,All customs procedure codes,1.82%,1.82%,1.82%
236,Portugal,Sao Tome and Principe,World,2021,Export,"Sausages and similar products, of meat, meat offal, blood or insects; food preparations based on these products","1,137,716.94",1137716.935000,1.70%,All modes of transport,True,All customs procedure codes,1.70%,1.70%,1.70%
